# Test EIA860 validation for old years

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Standard libraries
import logging
import sys
import os
import pathlib

# 3rd party libraries
import geopandas as gpd
import dask.dataframe as dd
from dask.distributed import Client
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
import seaborn as sns
import sqlalchemy as sa

# Local libraries
import pudl

In [3]:
# Enable viewing of logging outputs
logger=logging.getLogger()
logger.setLevel(logging.INFO)
handler = logging.StreamHandler(stream=sys.stdout)
formatter = logging.Formatter('%(message)s')
handler.setFormatter(formatter)
logger.handlers = [handler]

In [4]:
# Display settings
sns.set()
%matplotlib inline
mpl.rcParams['figure.dpi'] = 150
pd.options.display.max_columns = 100
pd.options.display.max_rows = 5

In [5]:
# Establish connection to pudl database
pudl_settings = pudl.workspace.setup.get_defaults()
pudl_engine = sa.create_engine(pudl_settings['pudl_db'])

In [6]:
# Change root directory to pytest test folder
PUDL_REPO = pathlib.Path("/Users/aesharpe/Desktop/Work/Catalyst_Coop/pudl/")
os.chdir(PUDL_REPO / "test")

# Import test file
import validate.eia_test

In [7]:
# Generate pudl_out_eia object -- change freq as desired for tests
pudl_out_eia = pudl.output.pudltabl.PudlTabl(
    pudl_engine=pudl_engine,
    #freq="AS",
    fill_fuel_cost=True,
    roll_fuel_cost=True,
    fill_net_gen=True,
)

In [8]:
# Test parameters -- change for each test function
row_params = (
    "df_name,cols", [
        ("plants_eia860", "all"),
        ("utils_eia860", "all"),
        ("pu_eia860", "all"),
        ("bga_eia860", "all"),
        ("own_eia860", "all"),
        ("gens_eia860", "all"),
        ("gen_eia923", "all"),
        ("gf_eia923", "all"),
        ("bf_eia923", "all"),
        ("frc_eia923", "all"),
    ]
)

def params_to_args(params):
    keys = params[0].split(',')
    dicts = []
    for args in params[1]:
        new_dict = {}
        for key,arg in zip(keys,args):
            new_dict[key] = arg
        dicts.append(new_dict)
    return dicts

row_args = params_to_args(row_params)

In [9]:
# Run test of desired function -- change out function as desired
validate.eia_test.test_no_null_cols_eia(
    pudl_out_eia=pudl_out_eia,
    live_pudl_db=True,
    **row_args[1]
)

### SQL Access

In [28]:
pudl_engine = sa.create_engine(pudl_settings["pudl_db"])

In [29]:
utils = pd.read_sql("utilities_eia860", pudl_engine)

In [30]:
utils

,id,utility_id_eia,report_date,street_address,city,state,zip_code,plants_reported_owner,plants_reported_operator,plants_reported_asset_manager,plants_reported_other_relationship,entity_type,attention_line,address_2,zip_code_4,contact_firstname,contact_lastname,contact_title,contact_firstname_2,contact_lastname_2,contact_title_2,phone_extension_1,phone_extension_2,phone_number_1,phone_number_2
0,1,63610,2019-01-01,30491 State Hwy 13,Waseca,MN,56093,True,None,None,None,Independent Power Producer,None,None,None,None,None,None,None,None,None,None,None,None,None
1,2,63609,2019-01-01,32625 Northfield Boulevard,Northfield,MN,55057,True,None,None,None,Independent Power Producer,None,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88856,88857,8,2004-01-01,2200 East Eldorado Street,Decatur,IL,62525,None,None,None,None,None,William D Coats,None,None,Scott,Market,Utilities Resource,William D,Coats,Utilities Manager,None,None,765-448-7123,765-448-7328
88857,88858,7,2004-01-01,2200 East Eldorado Street,Decatur,IL,62525,None,None,None,None,None,None,None,None,James A,Hoyt,Manager Energy Purchases,Jim,Cornelissen,"Technical Director, Americas",None,None,217-421-2503,217-423-4411
